# Step 1: Querying *aquagar* DB
* Defining *aquagar_data*
* Table 'aquagar' is queried looking for new rows in the last 24 hours.
    * Only the fields TIME_STAMP, SERIAL_NUM, PLATE_ID, TIME_POINT, TIME_STAMP, ROW, PRED are queried.

In [1]:
# Set the range in days
range = '1'

In [2]:
# Query the 'aquagar' table in KOAPredictions database
import mysql.connector

# Replace with your MySQL connection details
host =  '10.8.0.1'
username = 'pere'
password = 'Nemomola5'
database_name =  'KOAPredictions'

# Create a connection to the MySQL server
db_connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database_name
)

# Create a cursor to execute SQL commands
cursor = db_connection.cursor(dictionary=True)

try:
    # Query the full table with the specified range
    query = f"SELECT TIME_STAMP, SERIAL_NUM, PLATE_ID, TIME_POINT, TIME_STAMP, ROW, PRED, HUMAN_CHECKED FROM aquagar WHERE STR_TO_DATE(TIME_STAMP, '%Y-%m-%d %H:%i:%s') >= DATE_SUB(CURDATE(), INTERVAL {range} DAY)"
    cursor.execute(query)

    # Fetch all rows
    aquagar_data = cursor.fetchall()
    cursor.close()

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    db_connection.close()


# Step 2: Filtering logic
* For every (serial_num, plate_id, row) we *only look at the latest entry*. 
* Human_checked must be true.
* MISSING: if its time_point <= 24h, we should not consider it for email-sending (it will be considered at a later stage).

In [3]:
import pandas as pd
import json

df = pd.DataFrame(aquagar_data)

# Convert timestamp to datetime format
df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP'])

# Find the index of the latest timestamp for each plate_id
latest_indices = df.groupby(['SERIAL_NUM', 'PLATE_ID', 'ROW'])['TIME_STAMP'].idxmax()

# Create a new DataFrame with only the rows corresponding to the latest timestamps
latest_df = df.loc[latest_indices]

# Format the PRED column to dictionaries
latest_df['PRED'] = latest_df['PRED'].apply(lambda x: json.loads(x.decode('utf-8')))

# Filter by human_checked true
latest_df = latest_df[latest_df['HUMAN_CHECKED'] == True]
latest_df


,TIME_STAMP,SERIAL_NUM,PLATE_ID,TIME_POINT,ROW,PRED,HUMAN_CHECKED
2,2023-11-30,AA-202310-001,Non-readable,48h,1,"{'ahydrophila': 0, 'asalmonicida': 0, 'bcereus...",1
3,2023-11-30,AA-202310-001,Non-readable,48h,2,"{'ahydrophila': 0, 'asalmonicida': 0, 'bcereus...",1


# Step 3: Transforming *prediction* object
* We want to do two things:
    * Go from a *species-level* prediction to a *family-level* prediction.
    * We want icons for Low-Med-Hig.
    * QUESTION: Do we want an icon for NONE?

In [4]:
from utils import species2families, families2icons, families2microdiversity

# Create family-level predictions
latest_df['PRED_FAMILY'] = latest_df['PRED'].apply(species2families)
latest_df['PRED_FAMILY_ICONS'] = latest_df['PRED_FAMILY'].apply(families2icons)
latest_df['DIVERSITY_INDICATOR'] = latest_df['PRED_FAMILY'].apply(families2microdiversity)
latest_df

,TIME_STAMP,SERIAL_NUM,PLATE_ID,TIME_POINT,ROW,PRED,HUMAN_CHECKED,PRED_FAMILY,PRED_FAMILY_ICONS,DIVERSITY_INDICATOR
2,2023-11-30,AA-202310-001,Non-readable,48h,1,"{'ahydrophila': 0, 'asalmonicida': 0, 'bcereus...",1,"{'Vibrio': 10, 'Aeromonas': 0, 'Photobacterium...","{'Vibrio': '🟢', 'Aeromonas': '⚪️', 'Photobacte...",➖
3,2023-11-30,AA-202310-001,Non-readable,48h,2,"{'ahydrophila': 0, 'asalmonicida': 0, 'bcereus...",1,"{'Vibrio': 15, 'Aeromonas': 0, 'Photobacterium...","{'Vibrio': '🟠', 'Aeromonas': '⚪️', 'Photobacte...",➖


# Emailing
* Create the list *one_row_one_mail* where rows 1 & 2 are collapsed. That way we get 1 row = 1 email.

In [5]:
# List of dictionaries: every item in the list is a different email!
one_row_one_mail = []
for timestamp in latest_df['TIME_STAMP'].unique():
    timestamp_filtered_df = latest_df[latest_df['TIME_STAMP']==timestamp]
    if len(timestamp_filtered_df) == 2:
        timestamp, serial_num, plate_id, time_point = timestamp_filtered_df[['TIME_STAMP', 'SERIAL_NUM', 'PLATE_ID', 'TIME_POINT']].iloc[0].values
        family_pred_row1, family_pred_row2 = timestamp_filtered_df[timestamp_filtered_df['ROW']=='1']['PRED_FAMILY_ICONS'].values[0], timestamp_filtered_df[timestamp_filtered_df['ROW']=='2']['PRED_FAMILY_ICONS'].values[0]
        diversity_indicator_row1, diversity_indicator_row2 = timestamp_filtered_df['DIVERSITY_INDICATOR'].values[0], timestamp_filtered_df['DIVERSITY_INDICATOR'].values[1]
        row = {'TIME_STAMP': timestamp, 
               'SERIAL_NUM': serial_num, 
               'PLATE_ID':plate_id, 
               'TIME_POINT': time_point, 
               'FAMILY_PRED_ROW1': family_pred_row1, 
               'FAMILY_PRED_ROW2': family_pred_row2, 
               'DIVERSITY_INDICATOR_ROW1': diversity_indicator_row1,
               'DIVERSITY_INDICATOR_ROW2': diversity_indicator_row2}
        one_row_one_mail.append(row)

In [6]:
from utils import send_report_email, generate_html_tables, serial_numbers_to_email_addresses

In [8]:
for row in one_row_one_mail:
    send_report_email(email_receivers=[serial_numbers_to_email_addresses[row['SERIAL_NUM']]['email']], 
                      name_receiver=serial_numbers_to_email_addresses[row['SERIAL_NUM']]['name'],
                      timestamp = row['TIME_STAMP'], 
                      plate_id = row['PLATE_ID'], 
                      serial_num = row['SERIAL_NUM'],
                      html_tables=generate_html_tables(row['FAMILY_PRED_ROW1'], row['FAMILY_PRED_ROW2'], row['DIVERSITY_INDICATOR_ROW1'],row['DIVERSITY_INDICATOR_ROW2'])
                      )


In [7]:
# Testing
for row in one_row_one_mail:
    html_code = generate_html_tables(row['FAMILY_PRED_ROW1'], row['FAMILY_PRED_ROW2'], row['DIVERSITY_INDICATOR_ROW1'], row['DIVERSITY_INDICATOR_ROW2'])

html_code

'\n    <!DOCTYPE html>\n    <html lang="en">\n    <head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <style>\n        body {\n        font-family: Arial, sans-serif;\n        margin: 0;\n        padding: 0;\n        }\n\n        .container {\n        display: flex;\n        }\n\n        .left-column {\n        flex: 70%;\n        padding: 20px;\n        }\n\n        .right-column {\n        flex: 30%;\n        padding: 20px;\n        }\n\n        table {\n        width: 100%;\n        border-collapse: collapse;\n        margin-bottom: 20px;\n        }\n\n        th, td {\n        border: 1px solid #dddddd;\n        text-align: center;  /* Center align text */\n        padding: 8px;\n        }\n\n        th {\n        background-color: #f2f2f2;\n        }\n        \n        /* Style for the blank row */\n        .blank-row td {\n            border: none;\n        }\n\n        /* Other styles remain unchanged */\n\n    </st